## GRADIO_NCF_Recommendation

### 추천화 함수

In [6]:
import numpy as np
import pandas as pd
import sys
import tensorflow as tf

%cd /Users/raekkkky/Desktop/DNA/[24 1학기] 🏆승승장구/11주차/data
sys.path.append('/Users/raekkkky/Desktop/DNA/[24 1학기] 🏆승승장구/11주차/허깅스페이스/data')
print(sys.path)
print(tf.config.list_physical_devices('GPU'))

[Errno 2] No such file or directory: '/Users/raekkkky/Desktop/DNA/[24 1학기] 🏆승승장구/11주차/data'
/Users/raekkkky/Desktop/DNA/[24 1학기] 🏆승승장구/11주차
['/Users/raekkkky/Desktop/DNA/[24 1학기] 🏆승승장구/11주차', '/opt/homebrew/Cellar/python@3.11/3.11.9_1/Frameworks/Python.framework/Versions/3.11/lib/python311.zip', '/opt/homebrew/Cellar/python@3.11/3.11.9_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11', '/opt/homebrew/Cellar/python@3.11/3.11.9_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/lib-dynload', '', '/opt/homebrew/lib/python3.11/site-packages', '/Users/raekkkky/Desktop/DNA/[24 1학기] 🏆승승장구/11주차/data', '/Users/raekkkky/Desktop/DNA/[24 1학기] 🏆승승장구/11주차/허깅스페이스/data/', '/Users/raekkkky/Desktop/DNA/[24 1학기] 🏆승승장구/11주차/허깅스페이스/data']
[]


In [9]:
# rating 데이터 가져오기
rating_df=pd.read_csv('/Users/raekkkky/Desktop/DNA/[24 1학기] 🏆승승장구/11주차/허깅스페이스/data/rating_complete.csv')
print("Shape of the Dataset:",rating_df.shape)
rating_df.head(3)

Shape of the Dataset: (57633278, 3)


,user_id,anime_id,rating
0,0,430,9
1,0,1004,5
2,0,3010,7


In [10]:
# user와 애니메이션 별로 순서대로 0부터 ~~까지 순서대로 대응대도록 유저ID 인코딩 정렬.
# 이게 모델에 들어갈 user와 anime의 인덱스가됨
user_ids = rating_df["user_id"].unique().tolist() # 유저 아이디 가져오는거고
user2user_encoded = {x: i for i, x in enumerate(user_ids)} # 인덱스 초기화
user_encoded2user = {i: x for i, x in enumerate(user_ids)} # 인덱스 초기화 (반대)
rating_df["user"] = rating_df["user_id"].map(user2user_encoded) # 매핑
n_users = len(user2user_encoded) # 길이 출력
# user2user_encoded와 user_encoded2user의 차이는 서로 반대 버전이라는거임 user_userencoded2user는 인코딩된 값에서 원래의 user_id로 인코딩하는 것

# anime는 위와 동일함
anime_ids = rating_df["anime_id"].unique().tolist() # 애니 아이디 기준.. 작업 동일
anime2anime_encoded = {x: i for i, x in enumerate(anime_ids)} # 동일
anime_encoded2anime = {i: x for i, x in enumerate(anime_ids)} # 동일
rating_df["anime"] = rating_df["anime_id"].map(anime2anime_encoded) # 동일
n_animes = len(anime2anime_encoded) # 동일

print("Num of users: {}, Num of animes: {}".format(n_users, n_animes))
# print("Min rating: {}, Max rating: {}".format(min(rating_df['rating']), max(rating_df['rating'])))

Num of users: 310059, Num of animes: 16872


In [12]:
# 승태가 한 NCF[GMF + MLP] 모델 rating.csv 학습 후 가중치 및 모델에 필요한 파라미터들이 있는 결과만 가지고 옴!
# 승태가 이미 있는 데이터인 rating.csv에서 "사용자랑 애니메이션 별로 NCF를 활용한 평점 예측 훈련 결과" 모델
model = tf.keras.models.load_model('/Users/raekkkky/Desktop/DNA/[24 1학기] 🏆승승장구/11주차/허깅스페이스/data/anime_model.h5')

/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [13]:
# 애니메이션(아이템)과 애니메이션(아이템)과의 평점 학습 가중치 [사용자 간의 가중치도 했지만 사용 X]
def extract_weights(name, model):
    weight_layer = model.get_layer(name)
    weights = weight_layer.get_weights()[0]
    weights = weights / np.linalg.norm(weights, axis = 1).reshape((-1, 1))
    return weights

anime_weights = extract_weights('anime_embedding', model)
# user_weights = extract_weights('user_embedding', model)

# GMF와 MLP 모델 학습 결과인 user간의 학습 결과 가중치와 anime 간의 학습 결과 가중치를 학습한 모델에서의 임베딩 레이어로부터 가져옴
# 값의 범위가 다를 것이기에 -1~1 사이로 정규화함
# 하지만 우리는 아이템 기반 추천시스템이기에 anime_weughts만 있어도 됨

In [14]:
# 애니메이션 정보 가져오기 [메타 데이터 가져오기]
df_anime = pd.read_csv('/Users/raekkkky/Desktop/DNA/[24 1학기] 🏆승승장구/11주차/허깅스페이스/data/data_anime.csv', low_memory=True)
df_anime = df_anime.replace("Unknown", np.nan) # 결측치 처리
print("Shape of the Dataset:",df_anime.shape)
df_anime.head(5)

Shape of the Dataset: (17562, 35)


,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,...,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,...,229170.0,182126.0,131625.0,62330.0,20688.0,8904.0,3184.0,1357.0,741.0,1580.0
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1,"Sep 1, 2001",NaN,...,30043.0,49201.0,49505.0,22632.0,5805.0,1877.0,577.0,221.0,109.0,379.0
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,トライガン,TV,26,"Apr 1, 1998 to Sep 30, 1998",Spring 1998,...,50229.0,75651.0,86142.0,49432.0,15376.0,5838.0,1965.0,664.0,316.0,533.0
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),TV,26,"Jul 2, 2002 to Dec 24, 2002",Summer 2002,...,2182.0,4806.0,10128.0,11618.0,5709.0,2920.0,1083.0,353.0,164.0,131.0
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,冒険王ビィト,TV,52,"Sep 30, 2004 to Sep 29, 2005",Fall 2004,...,312.0,529.0,1242.0,1713.0,1068.0,634.0,265.0,83.0,50.0,27.0


In [15]:
# [1] rating에 anime_id와 anime에 anime_id 비교해서 같으면 영문명 이름 가져오고 아니면 error
# Fixing Names [이름 맞추고 데이터 수정] 
def getAnimeName(anime_id):
    try:
        name = df_anime[df_anime.anime_id == anime_id].eng_version.values[0]
        if name is np.nan:
            name = df_anime[df_anime.anime_id == anime_id].Name.values[0]
    except:
        print('error')

    return name

df_anime['anime_id'] = df_anime['MAL_ID']
df_anime["eng_version"] = df_anime['English name']
df_anime['eng_version'] = df_anime.anime_id.apply(lambda x: getAnimeName(x))

# 평점을 기반으로 정렬
df_anime.sort_values(by=['Score'],
               inplace=True,
               ascending=False,
               kind='quicksort',
               na_position='last')

# 정렬 (가져오고 싶은 것만)
df_anime = df_anime[["anime_id", "eng_version",
         "Score", "Genres", "Episodes",
         "Type", "Premiered", "Members"]]

# getAnimeName함수는 입력할 rating의 anime_id와 df_anime anime_id가 같으면 df_anime의 eng_name값을 반환하여 rating데이터에 없는 영문명을 반환하도록 하는 함수
# 그 이후는 df_anime의 변수명을 바꿔주고 df_anime의 score를 기준으로 정렬함
# 원하는 컬럼만 지정해서 새롭게 데이터프레임 저장

# [2] 애니 이름을 넣던, 애니 id를 넣던 추천할 수 있도록 제작
def getAnimeFrame(anime):
    if isinstance(anime, int): # 애니 id
        return df_anime[df_anime.anime_id == anime]
    if isinstance(anime, str): # 애니 영문명
        return df_anime[df_anime.eng_version == anime]

# [3] 시놉시스 데이터 원하는 데이터만 가져오기
cols = ["MAL_ID", "Name", "Genres", "sypnopsis"]
sypnopsis_df = pd.read_csv('/Users/raekkkky/Desktop/DNA/[24 1학기] 🏆승승장구/11주차/허깅스페이스/data/data_anime_with_synopsis.csv', usecols=cols)

# 이거는 [2] 함수랑 같은 기능임.
def getSypnopsis(anime):
    if isinstance(anime, int): # 애니 id
        return sypnopsis_df[sypnopsis_df.MAL_ID == anime].sypnopsis.values[0]
    if isinstance(anime, str): # 애니 영문명
        return sypnopsis_df[sypnopsis_df.Name == anime].sypnopsis.values[0]
        
# getSypnopsis함수는 입력할 rating의 anime_id와 sypnopsis_df의 MAL_id가 같으면 sypnopsis_df의 sypnopsis값을 반환하여 rating데이터에 없는 시놉시스(애니 요약된 설명)을 반환하도록 하는 함수

In [16]:
# 영화 3개 받고 3개 추천 [아이템 기반 추천]

def find_similar_animes_combined(anime_names, n=3, return_dist=False, neg=False):

    # try except문을 통해 try문속 오류, false가 뜨면 except문으로 가서 예외를 처리해줌
    # 반환 안될시 except 해주기 (점검)
    try:

        # getAnimeFrame 함수를 사용하여 각 애니메이션 제목에 대응하는 anime_id를 얻고, 이를 anime2anime_encoded 딕셔너리를 통해 인코딩된 인덱스로 변환하여 저장
        # 만들어놓은 리스트로 3개의 입력값들 저장
        encoded_indices = []
        input_anime_ids = []
        for name in anime_names:
            index = getAnimeFrame(name).anime_id.values[0]
            input_anime_ids.append(index)
            encoded_index = anime2anime_encoded.get(index)
            encoded_indices.append(encoded_index)


        # 사용자가 입력한 영화 3개를 기반으로 작업
        # 3개의 애니의 임베딩 벡터를 평균하여 하나의 결합 벡터를 생성하고, 이를 정규화함(여기서 임베딩 벡터는 기존의 애니 평점을 학습을 통해 애니메이션 별로 생성된 가중치 벡터를 뜻함)
        combined_weights = np.mean(anime_weights[encoded_indices], axis=0)
        combined_weights = combined_weights / np.linalg.norm(combined_weights)


        # 내적 함수를 통해 정규화한 임베딩 벡터와 전체 애니메이션 임베딩 벡터의 유사도를 구함
        dists = np.dot(anime_weights, combined_weights)

        # 유사도 행렬을 정렬함
        sorted_dists = np.argsort(dists)

         # 입력된 애니메이션의 수를 포함하여 n개의 가장 유사하거나 (neg=False) 가장 먼 (neg=True) 애니메이션을 선택함(+를 하는 이유는 기존의 3개가 제일 위에 뜰 것이기에 그걸 고려하고 3개 더 애니를 가져오기 위함임)
        n = n + len(input_anime_ids) # 영화 입력한 것을 제외하기 위해 개수 증가 [같은 것 추천 방지]

        if neg:
            closest = sorted_dists[:n]
        else:
            closest = sorted_dists[-n:]

        print('animes closest to {}'.format(anime_names)) # 이름 가장 비슷한거 출력 (ex. 나루토.. 만 입력 등)

        # 입력 파라미터 중 하나인 return_dist는 True로 지정하면 입력 임베딩 벡터의 유사도 행렬과 입력한 애니를 포함한 6개의 애니메이션을 반환함
        if return_dist:
            return dists, closest


        # df_anime 가져옴
        rindex = df_anime

        # 유사한 3개의 애니메이션 최종 결과 배열
        SimilarityArr = []

        #가장 유사한 3개의 애니를 for문 돌림
        for close in closest:

            # closet는 가장 유사한 애니들이 인코딩 된 anime_id로 가져왔기에 추천 시스템 출력 결과로 영문명과 장르를 추출하려면 기존의 anime_id로 디코딩이 필요함
            decoded_id = anime_encoded2anime.get(close)

            # 추천할 애니 3개의 디코딩한 id가 입력한 애니 3개 id랑 같으면 continue를 통해 for문을 스킵하게하여 출력 결과에 입력한 애니3개를 나오지 않도록 함
            if decoded_id in input_anime_ids:
                continue


            #getSypnopsis함수로 추천할 애니 3개의 시놉시스 가져옴
            sypnopsis = getSypnopsis(decoded_id)

            #getAnimeFrame함수로 추천할 애니 id에 해당하는 df_anime의 데이터프레임 값을 가져옴
            anime_frame = getAnimeFrame(decoded_id)

            # 가져온 데이터프레임의 이름과 장르를 추출
            anime_name = anime_frame.eng_version.values[0]
            genre = anime_frame.Genres.values[0]

            #유사도는 구한 유사도 행렬에서 인코딩 anime_id로 가져옴
            similarity = dists[close]

            #추천할 애니메이션 배열에 추가
            SimilarityArr.append({"anime_id": decoded_id, "name": anime_name,
                                  "similarity": similarity, "genre": genre,
                                  'sypnopsis': sypnopsis})

        #최종 추천 애니메이션 배열 dataframe으로 바꾸고 유사도를 기반으로 정렬
        Frame = pd.DataFrame(SimilarityArr).sort_values(by="similarity", ascending=False)

        #anime_id는 없어도 되니 빼고 지정한 n개의 값 출력하도록 반환
        return Frame.drop(['anime_id'], axis=1).head(n - len(input_anime_ids))  # Return top 'n' results excluding input animes

    #코드 오류 및 데이터에 입력하 애니 없을 떄 예외 처리
    except Exception as e:
        print('{}!, Not Found in Anime list'.format(anime_names))
        print(str(e))
        return pd.DataFrame()


# 1. 함수 정의 및 이름을 기반으로 한 임베딩 벡터 위치 찾기
# 2. 애니메이션에 대해서 리뷰를 기반으로 나온 가중치로 서로의 내적을 통해 유사도를 구함
# 3. 입력한 애니메이션과 유사한 애니메이션 찾기
# 4. 유사한 애니메이션 추려서 결과로 반환

In [17]:
# Example usage
anime_list = ['Sword Art Online', 'Attack on Titan Final Season', 'Steins;Gate']
find_similar_animes_combined(anime_list, n=3)

animes closest to ['Sword Art Online', 'Attack on Titan Final Season', 'Steins;Gate']


,name,similarity,genre,sypnopsis
3,Your Name.,0.701476,"Romance, Supernatural, School, Drama","suha Miyamizu, a high school girl, yearns to l..."
2,Your Lie in April,0.695971,"Drama, Music, Romance, School, Shounen",usic accompanies the path of the human metrono...
1,ERASED,0.687197,"Mystery, Psychological, Supernatural, Seinen","hen tragedy is about to strike, Satoru Fujinum..."


### 애니 이미지 크롤링 함수 : 근데 한국어 제목 영어 제목 충돌.. 사용 X

In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO

# 애니메이션 URL을 생성하는 함수
def get_anime_url(name, df):
    anime = df[df['eng_version'] == name]
    if not anime.empty:
        mal_id = anime['anime_id'].values[0]
        anime_name = anime['eng_version'].values[0].replace(' ', '_').replace(':', '_').replace('!', '_')
        return f"https://myanimelist.net/anime/{mal_id}/{anime_name}"
    else:
        print(f"{name}에 해당하는 애니메이션을 찾을 수 없습니다.")
        return None

# 이미지 URL을 추출하는 함수
def extract_image_url(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # 요청이 성공했는지 확인
    except requests.RequestException as e:
        print(f"페이지를 가져올 수 없습니다: {e}")
        return None

    soup = BeautifulSoup(response.text, 'html.parser')
    image_tag = soup.find('img', {'data-src': True})

    if image_tag:
        image_url = image_tag['data-src']
        return image_url
    else:
        print("이미지를 찾을 수 없습니다.")
        return None

# 이미지 URL에서 이미지를 표시하는 함수
def show_image(image_url):
    try:
        response = requests.get(image_url)
        response.raise_for_status()
        img = Image.open(BytesIO(response.content))
        plt.imshow(img)
        plt.axis('off')  # 축을 숨김
        plt.show()
    except requests.RequestException as e:
        print(f"이미지를 가져올 수 없습니다: {e}")

### GRADIO

In [19]:
import gradio as gr

def NCF_Recommendation(a, b, c):

    # 사용자 입력한 애니 이름 리스트화
    anime_list = [a, b, c]
    anime_result = find_similar_animes_combined(anime_list, n=3)
    
    # 애니 추천 3가지 (추천화 시스템 적용)
    result1 = anime_result.loc[3, 'name']
    result2 = anime_result.loc[2, 'name']
    result3 = anime_result.loc[1, 'name']

    # 추천된 애니 설명
    explain1 = anime_result.loc[3, 'sypnopsis']
    explain2 = anime_result.loc[2, 'sypnopsis']
    explain3 = anime_result.loc[1, 'sypnopsis']
 
    # 추천된 애니 메인 이미지 사진
    # image1 = "https://img.hankyung.com/photo/201712/AKR20171214060651033_01_i.jpg"
    # image2 = "https://m.media-amazon.com/images/M/MV5BYThlNWY5ZDgtYTIxNC00ZjdiLWJmNGUtMDFjMDlmZTAzOWFiXkEyXkFqcGdeQXVyNTM4NzAzNjc@._V1_FMjpg_UX1000_.jpg"
    # image3 = "https://m.media-amazon.com/images/M/MV5BYzJmZjZkMjQtZjJmZC00M2JkLTg5MzktN2FkOTllNTc5MmMzXkEyXkFqcGdeQXVyNjAwNDUxODI@._V1_.jpg"

    # 각각의 애니메이션 URL 생성
    url1 = get_anime_url(result1, df_anime)
    url2 = get_anime_url(result2, df_anime)
    url3 = get_anime_url(result3, df_anime)

    # 이미지 URL 추출
    image1 = extract_image_url(url1)
    image2 = extract_image_url(url2)
    image3 = extract_image_url(url3)
    
    return result1, explain1, image1, result2, explain2, image2, result3, explain3, image3

with gr.Blocks() as app:
    with gr.Row():
        a = gr.Textbox(label = "너의 최애 애니 첫 번째를 작성해봐!")
        b = gr.Textbox(label = "너의 최애 애니 두 번째를 작성해봐!")
        c = gr.Textbox(label = "너의 최애 애니 세 번째를 작성해봐!")

    with gr.Row():
        with gr.Column():
            img1 = gr.Image(label="1번째 애니 추천")
            output1 = gr.Textbox(label="1️⃣ 첫번째 애니 추천!")
            output2 = gr.Textbox(label="첫 번째 애니 설명", interactive = False)
        with gr.Column():
            img2 = gr.Image(label="2번째 애니 추천")
            output3 = gr.Textbox(label="2️⃣ 두번째 애니 추천!")
            output4 = gr.Textbox(label="두 번째 애니 설명", interactive = False)
        with gr.Column():
            img3 = gr.Image(label="3번째 애니 추천")
            output5 = gr.Textbox(label="3️⃣ 세번째 애니 추천!")
            output6 = gr.Textbox(label="세 번째 애니 설명", interactive = False)
    
    btn = gr.Button("추천을 받아봅시다!")
    
    btn.click(
        fn = NCF_Recommendation,
        inputs = [a, b, c],
        outputs = [output1, output2, img1, output3, output4, img2, output5, output6, img3]
    )

app.launch(share = True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://e12c332b6806a0d6c0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


animes closest to ['Sword Art Online', 'Attack on Titan Final Season', 'Steins;Gate']


In [12]:
# Sword Art Online
# Attack on Titan Final Season
# Steins;Gate

animes closest to ['Sword Art Online', 'Attack on Titan Final Season', 'Steins;Gate']
